In [43]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [44]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

In [45]:
# Reshape data to add channel dimension
X_train = np.reshape(X_train, (60000, 28, 28, 1))
X_test = np.reshape(X_test, (10000, 28, 28, 1))

In [46]:
# Jittering: Data augmentation by random cropping and flipping
data_augmentation = keras.Sequential([
    keras.layers.Resizing(28, 28),
    keras.layers.RandomCrop(28, 28),
    keras.layers.RandomFlip(mode='horizontal')
])
X_train = data_augmentation(X_train)

In [47]:
# Normalization: Scale pixel values from 0-255 to 0-1
X_train = tf.cast(X_train, dtype=tf.float32) / 255.0
X_test = tf.cast(X_test, dtype=tf.float32) / 255.0

In [48]:
# Define the CNN model
model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28,28,1)),
                             tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.MaxPooling2D(2, 2),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(32, activation = tf.nn.sigmoid),
                              tf.keras.layers.Dropout(0.2),
                              tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [49]:
#Summary of the architectural model
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6272)              0         
                                                                 
 dense_4 (Dense)             (None, 32)                200736    
                                                                 
 dropout_2 (Dropout)         (None, 32)               

In [50]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [52]:
# Train the model with early stopping
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.4249 - accuracy: 0.8882 - val_loss: 0.1777 - val_accuracy: 0.9513
Epoch 2/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1983 - accuracy: 0.9449 - val_loss: 0.1385 - val_accuracy: 0.9610
Epoch 3/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1553 - accuracy: 0.9548 - val_loss: 0.1202 - val_accuracy: 0.9638
Epoch 4/10
1875/1875 [==============================] - 38s 20ms/step - loss: 0.1315 - accuracy: 0.9599 - val_loss: 0.1264 - val_accuracy: 0.9612
Epoch 5/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.1170 - accuracy: 0.9646 - val_loss: 0.1096 - val_accuracy: 0.9664
Epoch 6/10
1875/1875 [==============================] - 39s 21ms/step - loss: 0.1063 - accuracy: 0.9680 - val_loss: 0.1131 - val_accuracy: 0.9663
Epoch 7/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0957 - accuracy: 0.9702 - val_loss: 0.1114 -

In [56]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 2s 7ms/step - loss: 0.1148 - accuracy: 0.9665
Test accuracy: 0.9664999842643738


In [54]:
# Make predictions
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

313/313 [==============================] - 2s 7ms/step


In [55]:
# Print classification report and confusion matrix
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print('\nClassification Report:\n', classification_report(y_test, y_pred))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.98      0.99      0.99      1135
           2       0.93      0.96      0.94      1032
           3       0.98      0.94      0.96      1010
           4       0.98      0.99      0.98       982
           5       0.93      0.97      0.95       892
           6       0.97      0.96      0.96       958
           7       0.97      0.95      0.96      1028
           8       0.98      0.97      0.97       974
           9       0.98      0.95      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


Confusion Matrix:
 [[ 966    1    1    0    1    0    8    1    0    2]
 [   0 1126    4    0    0    2    2    1    0    0]
 [   5    1  995    9    3    4    9    4    2    0]
 [   4    2   10  949    0   37    0 